<a href="https://colab.research.google.com/github/shikhar00778/toxic-comment-classification/blob/master/Toxic_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install git+https://github.com/kaushaltrivedi/fast-bert.git

  Cloning https://github.com/kaushaltrivedi/fast-bert.git to /tmp/pip-req-build-jr5o53bt
  Running command git clone -q https://github.com/kaushaltrivedi/fast-bert.git /tmp/pip-req-build-jr5o53bt
  Stored in directory: /tmp/pip-ephem-wheel-cache-c9dchn_e/wheels/53/11/ef/67d507c8f4f05b843a6dd2c4f6360acdee4c79679c4ca36bc6
Successfully built fast-bert


In [0]:
%%writefile /usr/local/lib/python3.6/dist-packages/fast_bert/metrics.py
import numpy as np
from torch import Tensor
from sklearn.metrics import roc_curve, auc, hamming_loss, accuracy_score
import pdb

# def accuracy(out, labels):
#     outputs = np.argmax(out, axis=1)
#     return np.sum(outputs == labels)

def accuracy(y_pred:Tensor, y_true:Tensor):

    outputs = np.argmax(y_pred, axis=1)
    return np.mean(outputs.numpy() == y_true.detach().cpu().numpy())

def accuracy_multilabel(y_pred:Tensor, y_true:Tensor, sigmoid:bool=True):
    if sigmoid: y_pred = y_pred.sigmoid()
    outputs = np.argmax(y_pred, axis=1)
    real_vals = np.argmax(y_true, axis=1)
    return np.mean(outputs.numpy() == real_vals.numpy())

def accuracy_thresh(y_pred:Tensor, y_true:Tensor, thresh:float=0.5, sigmoid:bool=True):
    "Compute accuracy when `y_pred` and `y_true` are the same size."
    if sigmoid: y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.byte()).float().mean().item()
#     return np.mean(((y_pred>thresh)==y_true.byte()).float().cpu().numpy(), axis=1).sum()


def fbeta(y_pred:Tensor, y_true:Tensor, thresh:float=0.3, beta:float=2, eps:float=1e-9, sigmoid:bool=True):
    "Computes the f_beta between `preds` and `targets`"
    beta2 = beta ** 2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = (y_pred>thresh).float()
    y_true = y_true.float()
    TP = (y_pred*y_true).sum(dim=1)
    prec = TP/(y_pred.sum(dim=1)+eps)
    rec = TP/(y_true.sum(dim=1)+eps)
    res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    return res.mean().item()

def roc_auc(y_pred: Tensor, y_true: Tensor):
    # ROC-AUC calcualation
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
        
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    return roc_auc["micro"]


Overwriting /usr/local/lib/python3.6/dist-packages/fast_bert/metrics.py


In [0]:
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import text,sequence
from tqdm import tqdm
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from fastai.train import Learner
from fastai.train import DataBunch
from fastai.callbacks import *
from fastai.metrics import *
import os
import random

import pytorch_pretrained_bert
from pytorch_pretrained_bert import BertModel
from pytorch_pretrained_bert.tokenization import BertTokenizer

from fast_bert.data import BertDataBunch
from fast_bert.learner import BertLearner
from fast_bert.metrics import accuracy , accuracy_multilabel

Using TensorFlow backend.


In [0]:

args = {
    "max_seq_length": 64,
    "do_lower_case": True,
    "train_batch_size": 32,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

In [0]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
!pip uninstall apex

Uninstalling apex-0.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/apex-0.1-py3.6-linux-x86_64.egg
Proceed (y/n)? y
  Successfully uninstalled apex-0.1


In [0]:
!rm -rf apex

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!python setup.py install --cuda_ext --cpp_ext

Cloning into 'apex'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 4467 (delta 38), reused 40 (delta 20), pack-reused 4394
Receiving objects: 100% (4467/4467), 8.67 MiB | 14.82 MiB/s, done.
Resolving deltas: 100% (2866/2866), done.
/content/apex
torch.__version__  =  1.1.0

Compiling cuda extensions with
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
from /usr/local/cuda/bin

running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-info/top_level.txt
writing manifest file 'apex.egg-info/SOURCES.txt'
writing manifest file 'apex.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_li

In [0]:
cd ..

/content


In [0]:
!ls

apex  sample_data


In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

05/26/2019 17:49:21 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [0]:
device = torch.device('cuda')

# check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1559149508&Signature=hZ8h%2BCpqFHDys56iBcLfz6SKnx4twthzyCngCYsMa4WAkEhhoASfcs9l2TcmAxLSMBkdPyaWZapiY5KwRXRcU6jan%2BphNLOGCRMzQ2FHqT8ia%2BFHfTNL1f57V4JIGfl01%2FeagyXlphuXi0U62O8UPuwoN6nplXKDBX9qBwoT3B%2Fd1YFUqr27ej7TegcMQ55S2xNlXMgddzCiBIkVOfsqEkLx2j2%2FaEL1RqLHfb0Sy4X%2FjrPaATbD2%2FWltXX0k20vzt%2BZDLfG%2FHjRAZCd7C0KU3moa8NPenqqbTVsTQj%2FbFBSrhlgJfrkGK1Gth%2BW8VdG1uNjKOd6ATePO%2BGnDX7QjA%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-toxic-comment-classification-challenge.zip" -O "jigsaw-toxic-comment-classification-challenge.zip" -c

--2019-05-26 17:21:23--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1559149508&Signature=hZ8h%2BCpqFHDys56iBcLfz6SKnx4twthzyCngCYsMa4WAkEhhoASfcs9l2TcmAxLSMBkdPyaWZapiY5KwRXRcU6jan%2BphNLOGCRMzQ2FHqT8ia%2BFHfTNL1f57V4JIGfl01%2FeagyXlphuXi0U62O8UPuwoN6nplXKDBX9qBwoT3B%2Fd1YFUqr27ej7TegcMQ55S2xNlXMgddzCiBIkVOfsqEkLx2j2%2FaEL1RqLHfb0Sy4X%2FjrPaATbD2%2FWltXX0k20vzt%2BZDLfG%2FHjRAZCd7C0KU3moa8NPenqqbTVsTQj%2FbFBSrhlgJfrkGK1Gth%2BW8VdG1uNjKOd6ATePO%2BGnDX7QjA%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-toxic-comment-classification-challenge.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55956017 (53M) [application/zip]
Saving to: ‘jigsaw-toxic-comment-classification

In [0]:
!unzip jigsaw-toxic-comment-classification-challenge.zip

Archive:  jigsaw-toxic-comment-classification-challenge.zip
  inflating: test_labels.csv         
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [0]:
!ls

apex						   test.csv
jigsaw-toxic-comment-classification-challenge.zip  test_labels.csv
labels.csv					   train.csv
sample_data					   valid.csv
sample_submission.csv


In [0]:
df = pd.read_csv('train.csv')

In [0]:
df_valid = df[120000:]

In [0]:
df_train = df[:120000]

In [0]:
df_train.head(1)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0


In [0]:
df_train = df_train.rename(index=str, columns={"comment_text": "text"})

In [0]:
df_valid = df_valid.rename(index=str, columns={"comment_text": "text"})

In [0]:
!rm -r train.csv

In [0]:
df_train.to_csv('train.csv' , index = False)

In [0]:
df_valid.to_csv('valid.csv' , index = False)

In [0]:
l = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']
df_label = pd.DataFrame(l)

In [0]:
df_label.to_csv('labels.csv' , index = False , header = None)

In [0]:
databunch = BertDataBunch('/content', '/content', tokenizer, 
                          train_file='train.csv', val_file='valid.csv',label_file='labels.csv',label_col=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate'],
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=True)

In [0]:
logger

<Logger __main__ (INFO)>

In [0]:
metrics = []
metrics.append({'name': 'accuracy_multilabel', 'function': accuracy_multilabel})

learner = BertLearner.from_pretrained_model(databunch, 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz', metrics, device, logger=logger, 
                                            finetuned_wgts_path=None, 
                                            is_fp16=args['fp16'], loss_scale=args['loss_scale'], 
                                            multi_gpu=multi_gpu,  multi_label=True)

05/26/2019 17:55:40 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
05/26/2019 17:55:40 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpnty1kvd1
05/26/2019 17:55:44 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size

In [0]:
learner.fit(1, lr=args['learning_rate'], 
            schedule_type="warmup_cosine_hard_restarts")

05/26/2019 22:28:55 - INFO - __main__ -   Loss after epoch 0 - 0.032352679665883385
05/26/2019 22:28:55 - INFO - __main__ -   Running evaluation


05/26/2019 22:30:50 - INFO - __main__ -   Eval results:
05/26/2019 22:30:50 - INFO - __main__ -     eval_loss = 0.04371506529848905
05/26/2019 22:30:50 - INFO - __main__ -     metrics = {'accuracy_multilabel': 0.9937833261732076}
05/26/2019 22:30:50 - INFO - __main__ -   --------------------------------------------------------------------------------


In [0]:
 df_test = pd.read_csv('test.csv')

In [0]:
texts = list(df_test['comment_text'])

In [0]:
predictions = learner.predict_batch([texts[0]])

In [0]:
texts[0:3] , texts[0:1] , texts[1:3]

(["Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,",
  '== From RfC == \n\n The title is fine as it is, IMO.',
  '" \n\n == Sources == \n\n * Zawe Ashton on Lapland —  /  "'],
 ["Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"],
 ['== From RfC == \n\n The title is fine as it is, IMO.',
  '" \n\n == Sources == \n\n * Zawe Ashton on Lapland —  /  "'])

In [0]:
len(texts)/22

6962.0

In [0]:
#predictions = learner.predict_batch([txt])
  toxic.append(predictions[0][0][1])
  obscene.append(predictions[0][1][1])
  insult.append(predictions[0][2][1])
  severe_toxic.append(predictions[0][3][1])
  identity_hate.append(predictions[0][4][1])
  threat.append(predictions[0][5][1])
  
  #print(toxic)
  #break

In [0]:
toxic = []
obscene = []
insult = []
severe_toxic = []
identity_hate = []
threat = []

import tqdm
for i in tqdm.tqdm(range(0,len(texts),22)):
  txt = texts[i:i+22]
  predictions = learner.predict_batch(txt)
  
  for j in range(22):
    toxic.append(predictions[j][0][1])
    obscene.append(predictions[j][1][1])
    insult.append(predictions[j][2][1])
    severe_toxic.append(predictions[j][3][1])
    identity_hate.append(predictions[j][4][1])
    threat.append(predictions[j][5][1])
  
  #print(toxic)
  #break

100%|██████████| 6962/6962 [06:35<00:00, 14.68it/s]


In [0]:
len(toxic)

153164

In [0]:
df_test['toxic'] = toxic

In [0]:
df_test['severe_toxic'] = severe_toxic
df_test['obscene'] = obscene
df_test['threat'] = threat
df_test['insult'] = insult
df_test['identity_hate'] = identity_hate


In [0]:
!rm -r submission_bert.csv

In [0]:
df_test.to_csv('submission_bert.csv' , index = False)

In [0]:
df_test = df_test.drop(['comment_text'] , axis = 1)

In [0]:
from google.colab import files
files.download('submission_bert.csv')

In [0]:
len(df_test)

153164

In [0]:
multi_gpu

False